In [1]:
# parameters
imgDim = [151, 151]

In [2]:
from models import simple_model
model = simple_model(17, imgDim)
print(model.summary())

Using TensorFlow backend.
C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomni\Anaconda3\e

In [3]:
from keras import backend as K

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
from keras.optimizers import Adadelta
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=Adadelta())

W1029 14:51:16.578461 14812 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [5]:
from dataGenerator import DataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, batch_size=128, inputDim=imgDim)
dataGenerator_val = DataGenerator(location_data, batch_size=128, inputDim=imgDim, isTrainingData=False)

In [6]:
from keras import callbacks
import time

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir='../tensorboard/' + time.strftime("%Y%m%d-%H%M%S"))

In [7]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    callbacks=[tensorboard],
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    verbose=1, epochs=100)

W1029 14:51:18.995302 14812 deprecation.py:323] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1029 14:51:22.404183 14812 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1029 14:51:22.405181 14812 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

Epoch 1/100
78/78 [==============================] - 55s 699ms/step - loss: 9.0051 - f1: 0.1065 - val_loss: 9.4051 - val_f1: 0.11

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[512,512,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_12/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[metrics/f1/Mean/_467]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[512,512,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_12/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.